# Introduction and Objective

======================================================

Group : 01 FTDS-026-HCK

======================================================

Objective :

## Background

## Dataset Description

Source: https://www.kaggle.com/datasets/orvile/x-ray-baggage-anomaly-detection/data

## Import Libraries

In [59]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from PIL import Image

from ultralytics import YOLO

In [60]:
import os
print(os.getcwd())


d:\Hactive\fase_2\finpro\p2-final-project-ftds-hck-026


In [61]:
# Memuat data load
data_path_train = 'dataset/train'
data_path_valid = 'dataset/valid'
data_path_test = 'dataset/test'

In [63]:
# Load dataset hanya untuk ambil class_names
ds_temp = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    image_size=(128, 128),
    batch_size=32
)
# Tampilkan mapping antara indeks dan nama kelas
for idx, class_name in enumerate(ds_temp.class_names):
    print(f"Class {idx} → {class_name}")


Found 6181 files belonging to 2 classes.
Class 0 → images
Class 1 → labels


In [64]:
# Check how many data in `train_path`, `valid_path`  and `test_path`
def check_path(path):
  labels = os.listdir(path)
  for label in labels:
    num_data = len(os.listdir(os.path.join(path, label)))
    print(f'Total Data - {label} : {num_data}')


In [65]:
# Menampilkan hasil data train
check_path(data_path_train)

Total Data - images : 6181
Total Data - labels : 6181


In [66]:
# Menampilkan hasil data valid
check_path(data_path_valid)

Total Data - images : 1766
Total Data - labels : 1766


In [ ]:
# import yaml

# path_yaml = 'data.yaml' # sesuaikan path ini dengan lokasi file yaml

# with open(path_yaml, 'r') as file:
#     data = yaml.safe_load(file)

# print(data)

# print("Train folder:", data['train'])
# print("Val folder:", data['val'])
# print("Test folder:", data['test'])
# print("Number of classes:", data['nc'])
# print("Class names:", data['names'])
# print("Roboflow workspace:", data.get('roboflow', {}).get('workspace'))




FileNotFoundError: [Errno 2] No such file or directory: 'data.yaml'